In [61]:
# Alphabet index for lookup
ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

class Rotor:
   # Default Wirings for rotors
    DEFAULT_WIRINGS = {
        'I': {'forward':'EKMFLGDQVZNTOWYHXUSPAIBRCJ',
              'backward':'UWYGADFPVZBECKMTHXSLRINQOJ'},
        'II':{'forward':'AJDKSIRUXBLHWTMCQGZNPYFVOE',
              'backward':'AJPCZWRLFBDKOTYUQGENHXMIVS'},
        'III':{'forward':'BDFHJLCPRTXVZNYEIWGAKMUSQO',
               'backward':'TAGBPCSDQEUFVNZHYIXJWLRKOM'},
    }
    # Default notches
    DEFAULT_NOTCHES = {
        'I':'Q',
        'II':'E',
        'III':'V',
    }
    def __init__(self, rotor_num, win_let, next_rotor=None, prev_rotor=None, ring_setting='A'):
          self.rotor_num = rotor_num
          self.n_rotor = next_rotor
          self.p_rotor = prev_rotor
          self.win = win_let.upper()
          self.wiring = self.DEFAULT_WIRINGS[rotor_num]
          self.notch = self.DEFAULT_NOTCHES[rotor_num]
          self.offset = ALPHABET.index(self.win)
          self.ring_offset = ALPHABET.index(ring_setting)

    def step(self):
        if self.n_rotor and self.win==self.notch:
            self.n_rotor.step()
        self.offset = (self.offset + 1)%26
        self.win = ALPHABET[self.offset]

    def encode_letter(self, index, forward=True, log=False):
        if type(index) == str and len(index) == 1:
            index = ALPHABET.index(index.upper())
        key = 'forward' if forward else 'backward'

        output_letter = self.wiring[key][(index + self.offset - self.ring_offset)%26]
        output_index = (ALPHABET.index(output_letter) - self.offset +self.ring_offset)%26
        if log:
           print('Rotor ' + self.rotor_num + ':\n OFFSET: ' + str(self.offset) + f"({ALPHABET[self.offset]}) \n RING_OFFSET: " + str(self.ring_offset)+'\n input = ' + ALPHABET[index] + f"({str(index)})"+ f"\n Process: WIRING[{key}][({self.offset} + {index} - {self.ring_offset}) % 26] = " +output_letter+ "(" + str(ALPHABET.index(output_letter)) +f')\n Output: ({str(ALPHABET.index(output_letter))} - {self.offset} + {self.ring_offset})%26 = {output_index} (' + ALPHABET[output_index] + ")")
        if self.n_rotor and forward:
            return self.n_rotor.encode_letter(output_index, forward, log=log)
        elif self.p_rotor and not forward:
            return self.p_rotor.encode_letter(output_index, forward, log=log)
        else:
            return output_index

    def change_setting(self, new_win_let):
        self.win = new_win_let.upper()
        self.offset = ALPHABET.index(self.win)


In [63]:
import re

class Enigma:
    def __init__(self, key='AAA', swaps=[], rotor_order=['I', 'II', 'III'], ring_setting=['A','A','A'], log=False):
        self.key = key
        self.rotor_order = rotor_order

        self.right_rotor = Rotor(rotor_order[0], key[0], ring_setting=ring_setting[0])
        self.middle_rotor = Rotor(rotor_order[1], key[1], self.right_rotor, ring_setting=ring_setting[1])
        self.left_rotor = Rotor(rotor_order[2], key[2], self.middle_rotor, ring_setting=ring_setting[2])

        # Default reflector
        self.reflector = {'A':'Y', 'B':'R', 'C':'U', 'D':'H', 'E':'Q', 'F':'S', 'G':'L', 'H':'D',
                       'I':'P', 'J':'X', 'K':'N', 'L':'G', 'M':'O', 'N':'K', 'O':'M', 'P':'I',
                       'Q':'E', 'R':'B', 'S':'F', 'T':'Z', 'U': 'C', 'V':'W', 'W':'V', 'X':'J',
                       'Y':'A', 'Z':'T'
                        }

        #Plugboard for swaps
        self.plugboard = {}
        for swap in swaps:
            self.plugboard[swap[0]] = swap[1]
            self.plugboard[swap[1]] = swap[0]

        self.middle_rotor.p_rotor = self.left_rotor
        self.right_rotor.p_rotor = self.middle_rotor
        self.log = log

    def __repr__(self):
        print('Initial Configuration Key: ' + self.key)
        print("Current Configuration Key: " + str(self.right_rotor.win) + str(self.middle_rotor.win) + str(self.left_rotor.win))
        return 'Key: ' + self.key

    def encipher(self, message):
        cipher = ''

        if bool(re.compile(r'[^a-zA-Z ]').search(message)):
            return 'Can only contain a-zA-Z'
        for letter in message.upper().strip():
            cipher += self.encode_decode_letter(letter)
        return cipher

    #Encipher, decipher process is identical
    def decipher(self, message):
        return self.encipher(message)

    def encode_decode_letter(self, letter):
        self.orig_letter = letter
        if letter == " ":
            return " "
        if bool(re.compile(r'[^a-zA-Z ]').search(letter)):
            return 'Can only contain a-zA-Z'
        # Swap to plugboard
        if letter in self.plugboard:
            letter = self.plugboard[letter.upper()]
        # Double step sequence
        if self.middle_rotor.win == self.middle_rotor.notch:
            self.middle_rotor.step()

        # Steps:
        # 1. Step rotor
        # 2. Pass to left rotor
        # 3. Feed to Reflektor
        # 4. Pass back to right rotor
        # 5. Plugboard again
        self.left_rotor.step()
        if self.log:
            print("Current Key: " + str(self.right_rotor.win) + str(self.middle_rotor.win) + str(self.left_rotor.win))
            print("INPUT: " + self.orig_letter)
            if self.orig_letter in self.plugboard:
              print("PLUGBOARD: " + letter.upper())

        left_pass = self.left_rotor.encode_letter(ALPHABET.index(letter.upper()), log=self.log)

        refl_output = self.reflector[ALPHABET[(left_pass)%26]]
        if self.log:
            print("REFLEKTOR: " + refl_output)
        final_letter = ALPHABET[self.right_rotor.encode_letter(ALPHABET.index(refl_output), forward=False, log=self.log)]

        if self.log:
          print("RESULT: " + final_letter)
          if final_letter in self.plugboard:
              print("PLUGBOARD: " + self.plugboard[final_letter])
          print("\n\n")

        if final_letter in self.plugboard:
            return self.plugboard[final_letter]
        else:
            return final_letter

    def set_rotor_position(self, pos_key):
        self.key = pos_key
        self.left_rotor.change_setting(self.key[2])
        self.middle_rotor.change_setting(self.key[1])
        self.right_rotor.change_setting(self.key[0])

In [64]:
import ipywidgets as widgets
from IPython.display import display

# Create widget objects
rotor1_widget = widgets.Dropdown(options=['I', 'II', 'III'], description='Rotor 1:', value ="I")
rotor2_widget = widgets.Dropdown(options=['I', 'II', 'III'], description='Rotor 2:', value = 'II')
rotor3_widget = widgets.Dropdown(options=['I', 'II', 'III'], description='Rotor 3:', value="III")
ring1_widget = widgets.Text(description='Ring Setting 1:', value="A")
ring2_widget = widgets.Text(description='Ring Setting 2:', value="A")
ring3_widget = widgets.Text(description='Ring Setting 3:', value="A")
rotors1_widget = widgets.Text(description='Rotor Seed 1:', value="A")
rotors2_widget = widgets.Text(description='Rotor Seed 2:', value="A")
rotors3_widget = widgets.Text(description='Rotor Seed 3:', value="A")
log_widget = widgets.ToggleButton(value=False, description='Enable Log')
text_widget = widgets.Textarea(description='Text:')
encrypt_button = widgets.Button(description='Encrypt')
decrypt_button = widgets.Button(description='Decrypt')
output_widget = widgets.Output()
swaps_widget = widgets.Text(description='Swaps (e.g., AB, CD):', value="AB,CD")

# Function to handle encryption button click event
def encrypt_button_clicked(b):
    with output_widget:
        output_widget.clear_output()
        # Get the selected values from the widgets
        rotor1 = rotor1_widget.value
        rotor2 = rotor2_widget.value
        rotor3 = rotor3_widget.value
        ring1 = ring1_widget.value.upper()
        ring2 = ring2_widget.value.upper()
        ring3 = ring3_widget.value.upper()
        rot1 = rotors1_widget.value.upper()
        rot2 = rotors2_widget.value.upper()
        rot3 = rotors3_widget.value.upper()
        log = log_widget.value
        text = text_widget.value

        # Create an instance of the Enigma machine with the user-selected settings
        swaps = [swap.strip().upper() for swap in swaps_widget.value.split(',')]  # Example list of swaps, replace with your own logic
        if "" in swaps:
          swaps = []
        enigma = Enigma(log=log, rotor_order=[rotor1, rotor2, rotor3], ring_setting=[ring1, ring2, ring3], swaps=swaps)
        enigma.set_rotor_position("".join([rot1, rot2, rot3]))

        print(text)
        # Encrypt the text
        encrypted_text = enigma.encipher(text)
        print("Encrypted Text:", encrypted_text)

# Function to handle decryption button click event
def decrypt_button_clicked(b):
    with output_widget:
        output_widget.clear_output()
        # Get the selected values from the widgets
        rotor1 = rotor1_widget.value
        rotor2 = rotor2_widget.value
        rotor3 = rotor3_widget.value
        ring1 = ring1_widget.value.upper()
        ring2 = ring2_widget.value.upper()
        ring3 = ring3_widget.value.upper()
        rot1 = rotors1_widget.value.upper()
        rot2 = rotors2_widget.value.upper()
        rot3 = rotors3_widget.value.upper()
        log = log_widget.value
        text = text_widget.value

        # Create an instance of the Enigma machine with the user-selected settings
        swaps = [swap.strip().upper() for swap in swaps_widget.value.split(',')] # Example list of swaps, replace with your own logic
        if "" in swaps:
          swaps = []
        enigma = Enigma(log=log, rotor_order=[rotor1, rotor2, rotor3], ring_setting=[ring1, ring2, ring3], swaps=swaps)
        enigma.set_rotor_position("".join([rot1, rot2, rot3]))

        # Decrypt the text
        print(text)
        decrypted_text = enigma.decipher(text)
        print("Decrypted Text:", decrypted_text)

# Assign click event handlers to the buttons
encrypt_button.on_click(encrypt_button_clicked)
decrypt_button.on_click(decrypt_button_clicked)

# Adjust the layout of the description widgets
rotor1_widget.layout.width = 'auto'
rotor2_widget.layout.width = 'auto'
rotor3_widget.layout.width = 'auto'
ring1_widget.layout.width = 'auto'
ring2_widget.layout.width = 'auto'
ring3_widget.layout.width = 'auto'
rotors1_widget.layout.width = 'auto'
rotors2_widget.layout.width = 'auto'
rotors3_widget.layout.width = 'auto'

# Display the widgets
widgets.VBox([
    widgets.HBox([rotor1_widget, rotor2_widget, rotor3_widget]),
    widgets.HBox([ring1_widget, ring2_widget, ring3_widget]),
    widgets.HBox([rotors1_widget, rotors2_widget, rotors3_widget]),
    swaps_widget,
    log_widget,
    text_widget,
    widgets.HBox([encrypt_button, decrypt_button]),
    output_widget
])